# Importing Modules and Libraries

We use `requests` and `BeautifulSoup` for web scraping

We use `hazm` for Persian text Processing.

In [42]:
import requests
from bs4 import BeautifulSoup

from hazm import *
from hazm.embedding import SentEmbedding
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import numpy as np
import re

# Web Scraping

news1: sports - football

news2: astronomy

news3: sports - football

## Getting data from URL

In [77]:
# NEWS URLs
NEWS1_url:str = r'https://www.tasnimnews.com/fa/news/1403/07/18/3174479/%D8%A2%D8%B2%D9%85%D9%88%D9%86-%D9%85%DB%8C-%D8%AF%D8%A7%D9%86%DB%8C%D9%85-%DA%86%D8%B7%D9%88%D8%B1-%D9%85%D9%82%D8%A7%D8%A8%D9%84-%D8%A7%D8%B2%D8%A8%DA%A9%D8%B3%D8%AA%D8%A7%D9%86-%D8%A8%D8%A7%D8%B2%DB%8C-%DA%A9%D9%86%DB%8C%D9%85'
NEWS2_url:str = r'https://www.tasnimnews.com/fa/news/1403/07/14/3172106/%D8%AC%D9%85%D8%B9-%D8%A2%D9%88%D8%B1%DB%8C-3-%D9%BE%D8%AA%D8%A7%D8%A8%D8%A7%DB%8C%D8%AA-%D8%AF%D8%A7%D8%AF%D9%87-%D9%87%D8%A7%DB%8C-%D9%85%D8%A7%D9%87%D9%88%D8%A7%D8%B1%D9%87-%D8%A7%DB%8C-%D8%AA%D9%88%D8%B3%D8%B7-%D8%B3%D8%A7%DB%8C%D8%AA-%D9%88%D8%B1%D8%A7%D9%85%DB%8C%D9%86'
NEWS3_url:str = r'https://www.tasnimnews.com/fa/news/1403/07/18/3174432/%D8%A8%DA%AF%DB%8C%D8%B1%DB%8C%D8%B3%D8%AA%D8%A7%DB%8C%D9%86-%D8%AF%D8%B1-%D9%BE%D8%A7%DB%8C%D8%A7%D9%86-%D9%81%D8%B5%D9%84-%D9%85%D9%86%DA%86%D8%B3%D8%AA%D8%B1%D8%B3%DB%8C%D8%AA%DB%8C-%D8%B1%D8%A7-%D8%AA%D8%B1%DA%A9-%D9%85%DB%8C-%DA%A9%D9%86%D8%AF'

# Requests to the server
news1_response = requests.get(NEWS1_url)
news2_response = requests.get(NEWS2_url)
news3_response = requests.get(NEWS3_url)


# Basically handling error
if news1_response.status_code != 200:
    raise Exception(f'The First NEWS Was not fetch with code: {news1_response.status_code}')

if news2_response.status_code != 200:
    raise Exception(f'The Second NEWS Was not fetch with code: {news2_response.status_code}')

if news3_response.status_code != 200:
    raise Exception(f'The Third NEWS Was not fetch with code: {news3_response.status_code}')

# Saving html contents in a dictionary
html_contents = {'n1': news1_response.text, 'n2': news2_response.text, 'n3':news3_response.text}

## Parsing HTML contents with soup
The text of each article is in a `div` with the `story` class.

In [78]:
extracted_texts = dict()

for idx, html_content in html_contents.items():
    soup = BeautifulSoup(html_content, 'html.parser')
    article = soup.find('div', class_='story').getText()
    article = article.replace('\u200c', ' ')
    article = article.replace('انتهای پیام/', '')
    extracted_texts[idx] = article

In [79]:
extracted_texts

{'n1': ' - اخبار ورزشی - به گزارش خبرنگار اعزامی خبرگزاری تسنیم از ازبکستان، سردار آزمون مهاجم تیم ملی فوتبال ایران در نشست خبری پیش از دیدار مقابل ازبکستان در مرحله سوم انتخابی جام جهانی 2026 در قاره آسیا، اظهار داشت: بازی سختی است، همیشه جلوی ازبکستان بازی سختی داشتیم، ولی ما به عنوان بازیکن می دانیم که چطور مقابل این تیم بازی کنیم. سه امتیاز این بازی برای ما اهمیت زیادی دارد و امیدواریم بتوانیم این سه امتیاز را به دست بیاوریم و راه مان را همین طور ادامه دهیم.قلعه نویی:از هر نظر آماده بازی با ازبکستانیم/کشور ما امن است\xa0در ادامه نشست خبری یک خبرنگار ازبکستانی از امیر قلعه نویی سؤالی درباره ضعف خط دفاعی تیم ملی ایران پرسید و گفت با این ضعف چه راهکاری برای مقابله با خط حمله ازبکستان اندیشیده اند که این سؤال با واکنش سردار آزمون مواجه شد و مهاجم تیم ملی کشورمان گفت: ببخشید اما با وجود اینکه شما این ضعف را می گویید، اما آخرین باری که ما را بردید کی بوده است؟!مهاجم تیم ملی فوتبال کشورمان در واکنش به کری خوانی یک بازیکن ازبکستان گفت: به عنوان بازیکن کری خوانی را دوست دارم، اما اگر منطقی 

# Processing Articles' Text

## Keyword Extraction

In [80]:
grammers = [
"""
NP:
        {<NOUN,EZ>?<NOUN.*>}    # Noun(s) + Noun(optional)

""",

"""
NP:
        {<NOUN.*><ADJ.*>?}    # Noun(s) + Adjective(optional)

"""
]
## you can also add your own grammer to be extracted from the text...

In [81]:
def extract_candidates(tagged, grammer):
    keyphrase_candidate = set()
    np_parser = nltk.RegexpParser(grammer)
    trees = np_parser.parse_sents(tagged)
    for tree in trees:
        for subtree in tree.subtrees(filter=lambda t: t.label() == 'NP'):  # For each nounphrase
            # Concatenate the token with a space
            keyphrase_candidate.add(' '.join(word for word, tag in subtree.leaves()))
    keyphrase_candidate = {kp for kp in keyphrase_candidate if len(kp.split()) <= 5}
    keyphrase_candidate = list(keyphrase_candidate)
    return keyphrase_candidate

In [82]:
pos_model_path = 'models/pos_tagger.model'
tagger = POSTagger(model = pos_model_path)

In [83]:
sent2vec_model_path = 'models/sent2vec/sent2vec-naab.model'
sent2vec_model = SentEmbedding(sent2vec_model_path)

In [84]:
# The dictionary
text_vectors = dict()

In [85]:
normalizer = Normalizer()
stop_words = stopwords_list()

for idx, text in extracted_texts.items():
    print(f'_______________ Processing NEWS {idx[1]} _______________')
    
    print('------Normalizing\n')
    # Normalizer    
    normalize_text = normalizer.normalize(text)
    
    print('------Tokenizing\n')
    # Tokenized
    tokenize_text = [word_tokenize(sent) for sent in sent_tokenize(normalize_text)]
    
    print('------Stop Word Removal\n')
    # Stop Word Removal    
    tokenized_text = []
    for sent in tokenize_text:
        sentence = []
        for word in sent:
            if word not in stop_words:
                sentence.append(word)
        tokenized_text.append(sentence)
    tokenize_text = tokenized_text
    
    print('------Extracting keyword candidates\n')
    token_tag_list = tagger.tag_sents(tokenize_text)
    
    print('------Extracting keyword candidates\n')
    # Extracting keyword candidates
    all_candidates = set()
    for grammer in grammers:
        all_candidates.update(extract_candidates(token_tag_list, grammer))



    print(f'------keywords for NEWS {idx[1]}\n')
    all_candidates = np.array(list(all_candidates))
    print('------------------------------')
    print(np.array(list(all_candidates)))
    print(f'Num candidates {len(list(all_candidates))}')
    print('------------------------------')
    
    print('------Embedding the text\n')
    # Embedding the keywords using sent2vec model
    candidates_concatinate = ' '.join(all_candidates)
    text_vector = sent2vec_model[candidates_concatinate]
    text_vectors[idx] = text_vector
    
    print(f'NEWS {idx[1]} vector: {text_vector.shape}')


_______________ Processing NEWS 1 _______________
------Normalizing

------Tokenizing

------Stop Word Removal

------Extracting keyword candidates

------Extracting keyword candidates

------keywords for NEWS 1

------------------------------
['نویی' 'ادامه خبری' 'فوتبال کشورمان' 'مقابله' 'گزارش خبرنگار' 'بازی'
 'فوتبال' 'بازیکن' 'قلعه' 'مقابله خط' 'امتیاز' 'فوتبال ایران'
 'بازیکن ازبکستان' 'تسنیم' 'دست' 'گزارش' 'مرحله' 'تیم ملی' 'اهمیت'
 'بازی سختی' 'خوانی' 'ایرانی\u200cها' 'کشورمان' 'سؤالی' 'ازبکستان' 'خبری'
 'عنوان' 'مرحله انتخابی' 'ایران' 'آزمون' 'حمله' 'واکنش' 'کشور امن' 'ادامه'
 'سال' 'سؤال واکنش' 'بازی ازبکستانیم' 'مهاجم تیم' 'امیر' 'نشان'
 'عنوان بازیکن' 'جام' 'زمین' 'آزمون مهاجم' 'امتیاز بازی' 'قلعه نویی' 'قوا'
 'احترام' 'تیم بازی' 'کشور' 'بازی ازبکستان' 'استرسی' 'خبرنگار ازبکستانی'
 'کری' 'خبرنگار اعزامی' 'جام جهانی' 'سردار' 'ضعف' 'چالش' 'خط دفاعی' 'تیم'
 'تیم\u200cها' 'قاره' 'کری خوانی' 'خط' 'ازبکستانیم' 'ضعف راهکاری' 'آسیا'
 '؟!' 'دوست' 'مان' 'اظهار' 'واکنش کری' 'خبرنگار' 

## Calculating Cosine Similarity

Football - Astronomy

In [86]:
text_vectors['n1'].dot(text_vectors['n2']) / (np.linalg.norm(text_vectors['n1'], ord=2) * np.linalg.norm(text_vectors['n2'], ord=2))

0.3460327

Football - Football

In [87]:
text_vectors['n1'].dot(text_vectors['n3']) / (np.linalg.norm(text_vectors['n1'], ord=2) * np.linalg.norm(text_vectors['n3'], ord=2))

0.36653438

Football - Astronomy

In [88]:
text_vectors['n2'].dot(text_vectors['n3']) / (np.linalg.norm(text_vectors['n2'], ord=2) * np.linalg.norm(text_vectors['n3'], ord=2))    

0.22850972

Football Football has the most cosine similarity, as we expected